In [132]:
import requests
import datetime
import toml
from IPython.display import Markdown
from collections import Counter

In [68]:
# Import the api configuration string data.
with open('blockchain_api_config.toml', 'r') as f:
    api_config = toml.loads(f.read())

<h1>At the time of your writing, what is the height of the most recent block mined and what is the
difficulty level?</h1>

In [12]:
def get_difficulty(_hash : str) -> int:
    _difficulty : int = 0
    for _letter in _hash:
        if _letter == "0":
            _difficulty += 1
        else:
            return _difficulty

In [19]:
latest_block = requests.get(api_config['latest_block']['root']).json()
hash_difficulty = requests.get(api_config['blockchain_api']['root'] + "q/getdifficulty").json()

height = latest_block['height']
difficulty = get_difficulty(latest_block['hash'])

Markdown(f"The height of the block is: {height}  \nThe difficulty-level is: {round(hash_difficulty/1000000000000, 2)} trillion.  \nThis is represented by {difficulty} initial '0' digits required for the hash of the block to be considered a successful hash.")

The height of the block is: 869858  
The difficulty-level is: 101.65 trillion.  
This is represented by 21 initial 0 requirement for the hash of the block to be mined.

<h1>What is the address of the miner of the block ? Can you unmask the identity of the
address?</h1>

In [43]:
def get_miner_addr(_tx : dict) -> str:
    for _out in _tx['out']:
        if _out['addr']:
            return str(_out['addr'])
        else:
            pass
        
    print("Error finding address for output.")

In [45]:
txs = latest_block['txIndexes']
tx = requests.get(api_config['single_transaction']['root'] + str(txs[0])).json()
miner_addr = None


if len(tx['inputs']) == 1:
    miner_addr = get_miner_addr(tx)
else:
    print("Error: Coinbase Transaction not the selected transaction")
    

Markdown(f"The Miner's Address is: {miner_addr}")

The Miner's Address is: 1PuJjnF476W3zXfVYmJfGnouzFDAXakkL4

<h1>How many transactions does it contain?</h1>

In [21]:
tx_num = len(txs)

Markdown(f"The number of transactions in the most recent block is: {tx_num}")

The number of transactions in the most recent block is: 2628

<h1>Which address has the most balance?</h1>

<i>Defining Functions</i>

In [102]:
def get_block_addresses(_current_block : dict, _in : bool) -> list:
    _addresses = []
    _txs = _current_block['txIndexes']
    
    if _in:
        for _tx in _txs[1:]:
            _req = requests.get(api_config['single_transaction']['root'] + str(_tx)).json()
            
            for _input in _req['inputs']:
                _addresses.append(_input['prev_out']['addr'])
                
    else:
        for _tx in _txs:
            _req = requests.get(api_config['single_transaction']['root'] + str(_tx)).json()
            
            for _out in _req['out']:
                
                if _out['value'] == 0:
                    continue
                if 'addr' in _out.keys():
                    _addresses.append(_out['addr'])
                else:
                    continue
                
    return _addresses        

In [103]:
def addresses_to_str(_addresses : list) -> str:
    _rtrn_str : str = ""
    
    for _addr in _addresses:
        
        if _rtrn_str == "":
            _rtrn_str = str(_addr)
            
        else:
            _rtrn_str += "|" + str(_addr)
            
    return _rtrn_str

In [104]:
def chunk_addresses(_addresses : list, _chunk_size : int = 50) -> list:
    for i in range(0, len(_addresses), _chunk_size):
        yield _addresses[i:i + _chunk_size]

In [105]:
def get_max_balance(_current_block : dict, _in : bool, _chunk_size : int = 50) -> dict:
    _addresses = get_block_addresses(_current_block, _in)
    
    _unique_addr = list(set(_addresses))
    rtrn_dict = {}
    max_balance = 0
    max_addr = ""
    
    for _addr_chunk in chunk_addresses(_unique_addr, _chunk_size):
        _address_str = addresses_to_str(_addr_chunk)
        
        balance_request = requests.get(api_config['balance']['root'] + _address_str)
        balance_data = balance_request.json()
        
        for _addr, _bal_info in balance_data.items():
            balance = _bal_info['final_balance']
            max_balance = max(max_balance, balance)
            
            if balance == max_balance:
                max_addr = _addr
    
    
    rtrn_dict[max_addr] = max_balance        
            
    return rtrn_dict

In [106]:
def get_exchange_rates() -> dict:
    return requests.get("https://blockchain.info/ticker").json()

In [107]:
def calc_currency_conversion(_exchange_data : dict, _currency : list, _btc : float) -> float:
    return _btc * int(_exchange_data[_currency[0]][_currency[1]])

In [108]:
def format_float_to_str(_float : float) -> str:
    return "{:,.2f}".format(_float)

<i>End of defining functions</i>

In [109]:
exchange_data = get_exchange_rates()

In [111]:
in_max = get_max_balance(latest_block, True, 50)

In [113]:
in_max_addr, in_max_balance = next(iter(in_max.items()))

In [128]:
in_max_btc = in_max_balance / 100000000
in_max_usd = calc_currency_conversion(exchange_data, ["USD", "15m"], in_max_btc)
in_max_usd_f = format_float_to_str(in_max_usd)


Markdown(f"Max btc balance of addresses who sent money in this block is: {in_max_btc}  "
         f"\nThe USD conversion equates to: ${in_max_usd_f}  "
         f"\nThe address with this value of bitcoin was: {in_max_addr}")

Max btc balance of addresses who sent money in this block is: 18208.77531143  
The USD conversion equates to: $1,546,616,957.40  
The address with this value of bitcoin was: bc1qm34lsc65zpw79lxes69zkqmk6ee3ewf0j77s3h

In [116]:
out_max = get_max_balance(latest_block, False, 50)

In [117]:
out_max_addr, out_max_balance = next(iter(out_max.items()))

In [129]:
out_max_btc = out_max_balance / 100000000
out_max_usd = calc_currency_conversion(exchange_data, ["USD", "15m"], out_max_btc)
out_max_usd_f = format_float_to_str(out_max_usd)


Markdown(f"Max btc balance of addresses who received money in this block is: {out_max_btc}  "
         f"\nThe USD conversion equates to: ${out_max_usd_f}  "
         f"\nThe address with this value of bitcoin was: {out_max_addr}")

Max btc balance of addresses who received money in this block is: 27490.96467139  
The USD conversion equates to: $2,335,027,557.26  
The address with this value of bitcoin was: bc1qr4dl5wa7kl8yu792dceg9z5knl2gkn220lk7a9

In [134]:
if out_max_btc > in_max_btc:
    print(f"Therefor, the address with the greatest balance is {out_max_addr} with {out_max_btc} BTC")
    
else:
    print(f"Therefor, the address with the greatest balance is {in_max_addr} with {in_max_btc} BTC")

<h1>Which address has most number of transactions?</h1>

In [119]:
total_addr = get_block_addresses(latest_block, True)
total_addr += get_block_addresses(latest_block, False)

In [137]:
addr_count = Counter(total_addr)

max_addr, max_count = addr_count.most_common(1)[0]

Markdown(f"The address with the most transactions in this block was: {max_addr}  "
         f"\nThey were involved in transactions (potentially as both senders and receivers) a total of {max_count} times in this block!")

The address with the most transactions in this block was: 1GrwDkr33gT6LuumniYjKEGjTLhsL5kmqC  
They were involved in transactions (potentially as both senders and receivers) a total of 556 times in this block!

<h1>When did this address become active? (first transaction on the network)</h1>

In [147]:
seen = requests.get(api_config['blockchain_api']['root'] + 'q/addressfirstseen/' + str(max_addr)).json()
seen_dt = datetime.datetime.utcfromtimestamp(seen)

Markdown(f"This address saw its first activity on the Bitcoin blockchain on "
         f"{seen_dt.strftime('%d/%m/%y')} at {seen_dt.strftime('%H:%M')}.")

This address saw its first activity on the Bitcoin blockchain on 06/08/21 at 06:32.